<div align="center">

# RIO Airbnb Project - Text Mining
**Latest Update:** *27th February 2023*

</div>

---

#### **Import required libraries & dataset**

In [2]:
import sys, os, re
sys.path.append(os.path.abspath("../"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import metrics

import airbnblib.cleaning as cln

In [ ]:
# Gets directly from dataset listings

In [9]:
# Gets the latest zipped cleaned version of the dataset
newest_listing_zip = os.listdir("../dataset_zipped/listings/")[-1]
listings_df = pd.read_csv(f"../dataset_zipped/listings/{newest_listing_zip}")

newest_listing_zip

'listings_cleaned_20230303.zip'

In [10]:
listings_df

,Unnamed: 0,id,name,description,neighborhood_overview,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,...,hair_dryer,hot_water,long_term_stays_allowed,smoke_alarm,refrigerator,kitchen,air_conditioning,tv,essentials,zip_code
0,0,183319.0,Panoramic Ocean View Venice Beach,Craftsmen style penthouse with ocean view from...,About as close to the beach as you can live in...,-15180,Serious yoga practice. Been studying for 25 y...,within a day,100%,100%,...,0,1,1,1,1,1,0,1,0,90291.0
1,1,51307.0,Spanish Bungalow Guest House LA CA. 30 plus ni...,"PRIVATE GUEST HOUSE The space Private, Guest h...","Local LA Community , shops and restaurants in...",-14867,"male , educated . Healthcare Professional\r\nC...",does not respond,NaN,NaN,...,0,1,1,1,1,1,1,1,1,90039.0
2,2,184314.0,Boho Chic Flat..Steps to Beach!,"Bright, airy, quiet 1 bdr located just steps f...",You are in one of the beach locations in Los A...,-15183,Health-Wellness Professional. Author. Passiona...,does not respond,NaN,NaN,...,1,0,1,1,0,1,0,1,1,90292.0
3,3,51498.0,Guest House With Its Own Entrance/Exit and Hot...,"Fully self-contained, separate structure, with...",We are close to Venice without the hassle of n...,-14868,Easy going hostess!! Enjoy your stay. \n\nI am...,within an hour,100%,88%,...,1,1,1,1,0,1,1,1,1,90066.0
4,4,109.0,Amazing bright elegant condo park front UPGRADED,"Unit upgraded with new bamboo flooring, brand...",NaN,-14057,"Paolo Privitera, CEO Evensi\n\nPaolo, MIT MBA ...",does not respond,NaN,NaN,...,1,0,1,1,0,1,1,1,1,90230.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45209,45209,26665862.0,"Queen Bed, Foam Mattress",Queen bed with a soft foam mattress in a priva...,"-Near Fairview Regional Park with hiking, biki...",-16991,NaN,within an hour,100%,99%,...,1,1,1,1,1,1,0,0,1,92626.0
45210,45210,26898605.0,SANITIZED Modern Zen Getaway | Relax & Replenish,Are you more than ready for a Local Getaway? N...,We are conveniently located just minutes from ...,-17725,Hi! My name is Isabella and I am a first gener...,within an hour,100%,98%,...,1,0,1,1,1,1,1,1,1,92626.0
45211,45211,26895703.0,"Simple, Clean, Modern One Bedroom Apartment",Welcome to Anaheim! This one bedroom suite ha...,We are in a safe and happy area. The street m...,-16728,It is nice finding that place where you can ju...,within an hour,98%,97%,...,1,1,1,1,1,1,1,1,1,92801.0
45212,45212,26954426.0,Oak Park House,"Good location, new carpet, kitchen and floorin...","Close to Brookside elementary, Medea creek and...",-16682,NaN,within a day,100%,NaN,...,1,1,1,1,0,1,1,1,1,91377.0


In [5]:
listings_df = listings_df.drop(['Unnamed: 0.1', 'Unnamed: 0', 'amenities'], axis=1)

In [11]:
from bs4 import BeautifulSoup

def utf8_only(text) -> str:
  remove_html = BeautifulSoup(text).get_text(separator=' ') if (type(text) == str) else ''
  remove_non_utf8 = bytes(remove_html, 'utf-8').decode('utf-8', 'ignore')
  
  return "".join(re.findall("[a-zA-Z ]", remove_non_utf8))



for col in ['name', 'description', 'neighborhood_overview']:
  listings_df[col] = listings_df[col].map(lambda x: utf8_only(x))

C:\Users\Tran Minh Khoi\AppData\Local\Temp\ipykernel_10212\1201725071.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  remove_html = BeautifulSoup(text).get_text(separator=' ') if (type(text) == str) else ''
C:\Users\Tran Minh Khoi\AppData\Local\Temp\ipykernel_10212\1201725071.py:4: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  remove_html = BeautifulSoup(text).get_text(separator=' ') if (type(text) == str) else ''


In [12]:
listings_df.head()

,Unnamed: 0,id,name,description,neighborhood_overview,host_since,host_about,host_response_time,host_response_rate,host_acceptance_rate,...,hair_dryer,hot_water,long_term_stays_allowed,smoke_alarm,refrigerator,kitchen,air_conditioning,tv,essentials,zip_code
0,0,183319.0,Panoramic Ocean View Venice Beach,Craftsmen style penthouse with ocean view from...,About as close to the beach as you can live in...,-15180,Serious yoga practice. Been studying for 25 y...,within a day,100%,100%,...,0,1,1,1,1,1,0,1,0,90291.0
1,1,51307.0,Spanish Bungalow Guest House LA CA plus nights,PRIVATE GUEST HOUSE The space Private Guest ho...,Local LA Community shops and restaurants in ...,-14867,"male , educated . Healthcare Professional\r\nC...",does not respond,NaN,NaN,...,0,1,1,1,1,1,1,1,1,90039.0
2,2,184314.0,Boho Chic FlatSteps to Beach,Bright airy quiet bdr located just steps from...,You are in one of the beach locations in Los A...,-15183,Health-Wellness Professional. Author. Passiona...,does not respond,NaN,NaN,...,1,0,1,1,0,1,0,1,1,90292.0
3,3,51498.0,Guest House With Its Own EntranceExit and Hot Tub,Fully selfcontained separate structure with it...,We are close to Venice without the hassle of n...,-14868,Easy going hostess!! Enjoy your stay. \n\nI am...,within an hour,100%,88%,...,1,1,1,1,0,1,1,1,1,90066.0
4,4,109.0,Amazing bright elegant condo park front UPGRADED,Unit upgraded with new bamboo flooring brand ...,,-14057,"Paolo Privitera, CEO Evensi\n\nPaolo, MIT MBA ...",does not respond,NaN,NaN,...,1,0,1,1,0,1,1,1,1,90230.0


In [8]:
from datetime import datetime

today = str(datetime.now()).split(" ")[0].replace('-', '')
listings_df.to_csv(f"../dataset/listings/listings_cleaned_{today}.csv", index=False)